In [1]:
import pandas as pd
import os
import json



In [2]:
headers=["sentence_id", "sentence", "phrase_start", "phrase_end", 'phrase', 'num_native_annotator', "num_non_native_annotator", "num_complex_marks_native", "num_complex_marks_non_native", "complex_binary", "complex_prob"]

datasets=["../data/cwishareddataset/traindevset/english/Wikipedia_Dev.tsv","../data/cwishareddataset/traindevset/english/Wikipedia_Train.tsv", "../data/cwishareddataset/testset/english/Wikipedia_Test.tsv"]
dataset_names = ['Wikipedia_dev', "Wikipedia_train", "Wikipedia_test"]

data = pd.read_csv(datasets[0], sep='\t', names=headers)
data

,sentence_id,sentence,phrase_start,phrase_end,phrase,num_native_annotator,num_non_native_annotator,num_complex_marks_native,num_complex_marks_non_native,complex_binary,complex_prob
0,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,4,8,tail,10,10,0,2,1,0.10
1,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,12,25,Epidexipteryx,10,10,6,3,1,0.45
2,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,31,35,bore,10,10,0,3,1,0.15
3,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,31,53,bore unusual vertebrae,10,10,0,1,1,0.05
4,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,36,53,unusual vertebrae,10,10,2,0,1,0.10
...,...,...,...,...,...,...,...,...,...,...,...
689,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,100,109,knowledge,10,10,0,2,1,0.10
690,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,74,80,person,10,10,0,0,0,0.00
691,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,95,99,true,10,10,0,0,0,0.00
692,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,113,122,purifying,10,10,7,8,1,0.75


In [3]:
def count_words(row):
    phrase = row['phrase']
    return len(phrase.split())

def get_words(data):
    data["complex_prob_str"] = data.apply(lambda row: format(row['complex_prob'], ".2f"), axis=1 )
    data['num_words'] = data.apply(lambda row: count_words(row), axis=1)
    words = data[data['num_words']==1]
    words['phrase'].str.lower()
    words['phrase'].str.strip()
    words = words.groupby('phrase')['complex_prob'].apply(max).reset_index()
    return words
    
words = get_words(data)
words[['phrase', 'complex_prob']].sort_values(['complex_prob'], ascending=False)


,phrase,complex_prob
472,thermodynamics,1.00
393,prototype,0.95
488,unearthed,0.95
351,monasteries,0.95
172,auspicious,0.90
...,...,...
207,commonly,0.00
206,common,0.00
203,collection,0.00
202,coldest,0.00


In [4]:
def get_text_and_metadata(words):

    text = " ".join(words['phrase'].tolist())

    phrases = words['phrase'].to_list()
    weights = words['complex_prob'].to_list()
    

    word_weights = words[['phrase', 'complex_prob']]
    
    return text, word_weights


print(get_text_and_metadata(words))



(' pharmacies A0 A9 AGL AMPAS Above Academy According Acherontia Adi Advaita Agreement Alcina Apis Appeal Arabic Armenian Arts Atman Australia Aviation Award Awards Awazu-kotei Beds Beijing Benjamin Best Bhakti Birthday Bradamante Brahman Britten Brown Bulgaria Chancellor Chernov China Community Companion Cosmic Counsel Court Covent Crozier Daohugou Death Devotion Division EIHL EU Earth English Epidexipteryx Eric Europe European February Festival GD Garden Gingerbread God Governor Gregory Ground Group Grégoire Hawk Honours House However Hungary IR Inner Institute Isvara January Japan Japanese Jiva Jomon Jupiter Jurassic Jōmon Karma Kelvin Level MSL Mahavakya Main-sequence Makar Manchester May Maya Mean Medicine Melbourne Melisso Merit Middle Mongolia Most Motion Nature Nicopolis Notes October Officer Opera Order Paleoanthropology Paleontology Paramarthika Pears Perceptible Peter Physically Picture Poland Queen Rankine Regions Romania Royal Ruggiero Ruin Satyam Sciences Sea Self Shankar

In [5]:
directory = "../data/evaluation/" 

if not os.path.exists(directory):
    os.makedirs(directory)

texts, wordsets = [],[]
    
for name, dataset in zip(dataset_names, datasets):
    data = pd.read_csv(dataset, sep='\t', names=headers)
    words = get_words(data)
    text, wordset = get_text_and_metadata(words)

    texts.append(text)
    wordsets.append(wordset)
    
    with open(directory+name+".txt", "w")as f:
        f.write(text)
        
    wordset.to_csv(path_or_buf=directory+name+".csv")


In [30]:

from utils.output_utils.create_df_from_doc import create_df_from_doc
from pipeline.create_pipeline import create_pipeline

pipeline = create_pipeline()
doc = pipeline(texts[0])
df = create_df_from_doc(doc)

print(texts[0])

519 overall words
 pharmacies A0 A9 AGL AMPAS Above Academy According Acherontia Adi Advaita Agreement Alcina Apis Appeal Arabic Armenian Arts Atman Australia Aviation Award Awards Awazu-kotei Beds Beijing Benjamin Best Bhakti Birthday Bradamante Brahman Britten Brown Bulgaria Chancellor Chernov China Community Companion Cosmic Counsel Court Covent Crozier Daohugou Death Devotion Division EIHL EU Earth English Epidexipteryx Eric Europe European February Festival GD Garden Gingerbread God Governor Gregory Ground Group Grégoire Hawk Honours House However Hungary IR Inner Institute Isvara January Japan Japanese Jiva Jomon Jupiter Jurassic Jōmon Karma Kelvin Level MSL Mahavakya Main-sequence Makar Manchester May Maya Mean Medicine Melbourne Melisso Merit Middle Mongolia Most Motion Nature Nicopolis Notes October Officer Opera Order Paleoanthropology Paleontology Paramarthika Pears Perceptible Peter Physically Picture Poland Queen Rankine Regions Romania Royal Ruggiero Ruin Satyam Sciences 

In [28]:
df.sort_values(by="rel. frequency").head(50)

df['token']

159   NaN
284   NaN
294   NaN
288   NaN
5     NaN
       ..
200   NaN
224   NaN
232   NaN
302   NaN
309   NaN
Name: token, Length: 325, dtype: float64

In [31]:
def evaluate_ranking(reference_df, df_under_test):
    df_under_test['token'].str.lower()

    res = reference_df.merge(df_under_test, "outer", left_on="phrase", right_on="token")

    res["ranking_error"] = (
        abs(res["complex_prob"] - res["ranking"]) * res["complex_prob"]
    )

    return res

In [32]:
df_res = evaluate_ranking(wordsets[0], df)
df_res.sort_values(by='complex_prob', axis=0, ascending=False, na_position="last")



,phrase,complex_prob,token,lemma,appearance,difficulty,rel. frequency,ranking,cluster,example phrase,ranking_error
472,thermodynamics,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,prototype,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
488,unearthed,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,monasteries,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,auspicious,0.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
828,NaN,NaN,oviraptorosaurs,oviraptorosaur,1.0,8.0,NaN,NaN,NaN,normally number numerals objects occasions off...,NaN
829,NaN,NaN,pygostyle,pygostyle,1.0,8.0,NaN,NaN,NaN,provide purifying pygostyle reached readily re...,NaN
830,NaN,NaN,rectrices,rectrix,1.0,8.0,NaN,NaN,NaN,record rectrices red reference referred refers...,NaN
831,NaN,NaN,turīya,turīya,1.0,8.0,NaN,NaN,NaN,traditionally true turīya tutor types ultimate...,NaN


In [34]:
df_res.head(500)

,phrase,complex_prob,token,lemma,appearance,difficulty,rel. frequency,ranking,cluster,example phrase,ranking_error
0,pharmacies,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A9,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGL,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AMPAS,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
495,vanes,0.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,various,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,vary,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,vertebrae,0.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
